In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import sklearn
import os
import cv2

from sklearn.model_selection import train_test_split
import skimage
from skimage import transform,util


In [ ]:
data_dir = os.getcwd() + "/PokemonData/"

# array of 149 pokemon's images (list of 149 lists of x# of img arrays)
allpoke = []
poke_classes = []

# for svg images
# from cairosvg import svg2png

# read in all images to 149 sublists
img_count = 0
for subdir in sorted(os.listdir(data_dir)):
    if subdir == ".DS_Store":
        continue
    path = data_dir + subdir + "/"
    #print(path)
    poke_classes.append(subdir)
    poke_imgs = []
    for image in os.listdir(path):
        img = cv2.imread(path+image)
        #print(image)
        if image.endswith(('.svg', '.DS_Store')): # just skip it for now?
            print("FUCKKKKKKKK") 
            #I = svg2png(url = path+image)
            #nparr = np.fromstring(I, np.uint8)
            #img = cv2.imdecode(image, cv2.IMREAD_UNCHANGED)
            continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) #convert it to RGB channel
        
        # normalize color values and resize before adding
        norm_img = img/255
        resize_img = transform.resize(norm_img,(300,300))
        poke_imgs.append(resize_img)
        img_count+=1
    allpoke.append(poke_imgs)
    
plt.imshow(allpoke[0][0])
plt.colorbar()
plt.figure()

FUCKKKKKKKK
FUCKKKKKKKK
FUCKKKKKKKK
FUCKKKKKKKK
FUCKKKKKKKK


In [ ]:

def split_train_test(images, testsize): # return list of image arrays
    training_poke = []
    testing_poke = []
    for i in images:
        train_set, test_set = train_test_split(i, test_size=testsize, shuffle=True)
        training_poke.append(train_set)
        testing_poke.append(test_set)
    return training_poke, testing_poke

# flip/rotate/distort/blur/noise/resize

def aug_set(train_poke):
    tot_aug_imgs = 0
    aug_train = []
    for poke in train_poke:
        aug_poke = []
        for i in poke:
            aug_poke.append(i)
            # horiz flip and rotate 45 deg both ways
            hflip = np.fliplr(i)
            aug_poke.append(hflip)
#             counter_rot_hflip = transform.rotate(hflip, angle=45)
#             aug_poke.append(counter_rot_hflip)
#             rot_hflip = transform.rotate(hflip, angle=-45)
#             aug_poke.append(rot_hflip)
            # vert flip and rotate 45 deg both ways
            vflip = np.flipud(i)
            aug_poke.append(vflip)
            counter_rot_vflip = transform.rotate(vflip, angle=45)
            aug_poke.append(counter_rot_vflip)
            rot_vflip = transform.rotate(vflip, angle=-45)
            aug_poke.append(rot_vflip)
            # noise, blur
            noisy = util.random_noise(i)
            aug_poke.append(noisy)
            blur = cv2.GaussianBlur(i, (11,11),0)
            aug_poke.append(blur)
            tot_aug_imgs+=9
        aug_train.append(aug_poke)
    print(tot_aug_imgs)
    return aug_train

print("num of abra images before aug")
print(img_count)
train_set, test_set = split_train_test(allpoke, .2)
print("num of abra images after aug")
aug_train_set = aug_set(train_set)
# print(len(train_set[0][0]))

# print(len(aug_train_set[0]))

In [ ]:
print(len(train_set[0]))
print(len(aug_train_set[0]))

imge = aug_train_set[0][:7]
for i in imge:
    plt.imshow(i)
    plt.figure()
    

for poke in aug_train_set:
    poke = [transforms.ToTensor(x) for x in poke]
for poke in test_set:
    poke = [transforms.ToTensor(x) for x in poke]

# Put into a Dataloader using torch library
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size =64, shuffle=True)

In [ ]:
# transformations = transforms.Compose([
#     transforms.ToTensor(),
# #     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# ])


# move the input and model to GPU for speed if available
device = torch.device("cuda" if torch.cuda.is_available() 
                                  else "cpu")
model = models.mobilenet_v2(pretrained=True)
print(model)